# Exploratory Analysis and PoC for creating a sentiment analysis app for the EPL Teams

In [9]:
import tweepy
import json
import pandas as pd
from datetime import datetime
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
# nltk.download('vader_lexicon') # Need this to run SIA()
# need to install pyarrow

In [34]:
df_query_list=pd.read_csv('query_list.csv')

In [35]:
df_query_list=df_query_list.head() #debug for first 5 rows

In [31]:
with open("C:/Users/William Jiang/Documents/credentials.json") as f:
    d = json.load(f)
    cred_json=d["twitter_api"]
    

# Authenticate to Twitter
auth = tweepy.OAuthHandler(cred_json["CONSUMER_KEY"], cred_json["CONSUMER_SECRET"])
auth.set_access_token(cred_json["ACCESS_TOKEN"], cred_json["ACCESS_TOKEN_SECRET"])

# Create API object
api = tweepy.API(auth)

# # Create a tweet
# api.update_status("Hello Tweepy2")

In [32]:
def polarity(row):
    #Does the sentiment analysis stuff
    sia = SIA()
    polarity=sia.polarity_scores(row)
    return polarity['compound']

In [ ]:
import os
arr = os.listdir('output')

In [49]:
for index,row in df_query_list.iterrows():
    #initialize the search_id_list
    try:
        search_id_list=[]
        for file in arr:
            if ".parquet" in file and row['file_name'] in file and row['category'] in file:
                search_id=file.split('-')[2]
                search_id_list.append(search_id)

        last_since_id=max(search_id_list)

        #Get Search Results
        search_results=api.search(q=row['search'], lang="en", count=15,since_id=last_since_id) #since_id neeeded here at some point
        rows_list=[]
        for tweet in search_results:
            dict1 = {}
            dict1['twitter_id']=tweet.id
            dict1['username']=tweet.user.name
            dict1['screenname']=tweet.user.screen_name
            dict1['text']=tweet.text
            dict1['created_at']=tweet.created_at
            dict1['retweet_count']=tweet.retweet_count
            dict1['location']=tweet.user.location
            dict1['followers']=tweet.user.followers_count
            dict1['friends']=tweet.user.friends_count
            rows_list.append(dict1)

        #output to dataframe and then to snappy.parquet form
        df=pd.DataFrame(rows_list) 
        file_dir='output/'
        df['polarity']=df['text'].apply(polarity)
        to_csv_timestamp = datetime.today().strftime('%Y%m%d_%H%M%S')
        max_twitter_id=max(df['twitter_id'])
        df.to_parquet(file_dir+row['category'] +'-'+ row['file_name'] +'-'+ str(max_twitter_id) + '-'+ to_csv_timestamp + '.snappy.parquet',compression='SNAPPY')
        df.to_csv(file_dir+row['category'] +'-'+ row['file_name'] +'-'+ str(max_twitter_id) + '-'+ to_csv_timestamp + '.csv') #debugging
    except Exception as e:
        print(e)

In [44]:
df_query_list

,file_name,group_name,category,search
0,Arsenal,Arsenal FC,EPL,#Arsenal OR #AFC OR #COYG
1,TottenhamHotspur,Tottenham Hotspur FC,EPL,#Tottenham OR #THFC OR #SPURS
2,ManchesterCity,Manchester City FC,EPL,#ManchesterCity OR #ManCity OR #MCFC
3,LeicesterCity,Leicester City FC,EPL,#LeicesterCity OR #LCFC
4,CrystalPalace,Crystal Palace FC,EPL,#CrystalPalace OR #CPFC


In [50]:
df

,twitter_id,username,screenname,text,created_at,retweet_count,location,followers,friends,polarity
0,1279290090099892224,Kevin Harper,kevshat,We've previewed today's #PremierLeague matches...,2020-07-04 05:45:00,0,Luton,1296,1429,0.0000
1,1279289838450151424,Profit Booster UK,ProfitBoosterUK,"Leicester City v Crystal Palace \n\nPreview, P...",2020-07-04 05:44:00,0,,41808,9910,0.6369
